In [1]:
import pandas as pd
import scipy
from sklearn import *

from SimpleCountVectorizer import *

# Count Vectorizer

In [3]:
train_df = pd.read_csv("C:/Users/toti/OneDrive/Jordi/.Master - Data Science UB/2nd SEMESTER/Natural Language Processing/course2020-master/quora-question-pairs/input/train.csv")

train_df, test_df = sklearn.model_selection.train_test_split(train_df, test_size=0.1, random_state=123)

In [4]:
train_df.shape, test_df.shape

((363861, 6), (40429, 6))

In [5]:
def cast_list_as_strings(mylist):
    """
    return a list of strings
    """
    assert isinstance(mylist, list), f"the input mylist should be a list it is {type(mylist)}"
    
    # return list(map(lambda x: str(x), all_questions)) # Slower
    return [str(s) for s in mylist]

In [6]:
all_questions = cast_list_as_strings(list(train_df.loc[:, 'question1'])+list(train_df.loc[:, 'question2']))
print(set(type(x).__name__ for x in all_questions))

{'str'}


In [7]:
count_vect = SimpleCountVectorizer()
count_vect.fit(all_questions)

SimpleCountVectorizer(doc_cleaner_func=None, doc_cleaner_pattern='[^a-zA-Z]',
           dtype=<class 'numpy.float32'>, min_word_counts=1,
           token_pattern='(?u)\\b\\w\\w+\\b', tokenizer_func=None,
           word_transformer_func=None)

In [8]:
def get_features_from_df(df, count_vectorizer):
    """
    returns a sparse matrix containing the features build by the count vectorizer.
    Each row should contain features from question1 and question2.
    """
    q1_casted =  cast_list_as_strings(list(df["question1"]))
    q2_casted =  cast_list_as_strings(list(df["question2"]))
        
    q1 = count_vectorizer.transform(q1_casted)
    q2 = count_vectorizer.transform(q2_casted)
    
    X_q1q2 = scipy.sparse.hstack((q1,q2))
        
    return X_q1q2

In [51]:
X_tr_q1q2 = get_features_from_df(train_df,count_vect)
X_te_q1q2  = get_features_from_df(test_df, count_vect)

X_tr_q1q2.shape, train_df.shape, test_df.shape, X_te_q1q2.shape

((363861, 151608), (363861, 5), (40429, 6), (40429, 151608))

In [57]:
# y_train = train_df["is_duplicate"].values
# y_test = test_df["is_duplicate"].values
len(y_train)

291088

In [55]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

param_grid = {
        'min_child_weight': [1, 5, 10],
        'max_depth': [3, 4, 5],
        'eval_metric': ["logloss"]
        }
    
d_train = xgb.DMatrix(X_tr_q1q2, label=y_train)
d_valid = xgb.DMatrix(X_te_q1q2, label=y_test)

watchlist = [(d_train, 'train'), (d_valid, 'valid')]

bst = xgb.train(param_grid, d_train, 400, watchlist, early_stopping_rounds=50, verbose_eval=10)

XGBoostError: [19:43:11] C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:65: Check failed: preds.Size() == info.labels_.Size() (363861 vs. 291088) : labels are not correctly providedpreds.size=363861, label.size=291088

In [ ]:
# logistic = sklearn.linear_model.LogisticRegression(solver="liblinear")
# logistic.fit(X_tr_q1q2, y_train)

In [11]:
def get_mistakes(clf, df, X_q1q2, y):

    predictions = clf.predict(X_q1q2)    
    incorrect_predictions = predictions!=y
    incorrect_indices = np.where(incorrect_predictions)[0]
    
    if np.sum(incorrect_predictions)==0:
        print("no mistakes in this df")
    else:
        return incorrect_indices, predictions
    
def print_mistake_k(k, mistake_indices, predictions):
    print(train_df.iloc[mistake_indices[k]].question1)
    print(train_df.iloc[mistake_indices[k]].question2)
    print("true class:", train_df.iloc[mistake_indices[k]].is_duplicate)
    print("prediction:", predictions[mistake_indices[k]])
    
def print_mistake_k_and_tokens(k, mistake_indices, predictions,
                               X_q1q2, count_vect, clf):
    q1 = train_df.iloc[mistake_indices[k]].question1
    q2 = train_df.iloc[mistake_indices[k]].question2
    
    print(q1)
    print(sorted(count_vect.tokenize(q1)))
    print("")
    print(q2)
    print(sorted(count_vect.tokenize(q2)))
    print("")
    print("true class:", train_df.iloc[mistake_indices[k]].is_duplicate)
    print("prediction:", predictions[mistake_indices[k]])
    print("")
    print("Probability vector: [P(0|x), P(1|x)]:")
    print(clf.predict_proba(X_q1q2)[mistake_indices[k],:])


In [12]:
mistake_indices, predictions = get_mistakes(logistic,train_df, X_tr_q1q2, y_train)

In [13]:
print_mistake_k(0, mistake_indices, predictions)

What are the best places to visit in or near Juneau, Alaska?
What are the best places to visit in Alaska and why?
true class: 0
prediction: 1


In [14]:
print_mistake_k_and_tokens(10, mistake_indices, predictions,
                           X_tr_q1q2, count_vect, logistic)

Why do men like women's feet?
['do', 'feet', 'like', 'men', 'why', 'women']

Why do men like womens feet?
['do', 'feet', 'like', 'men', 'why', 'womens']

true class: 1
prediction: 0

Probability vector: [P(0|x), P(1|x)]:
[0.55506132 0.44493868]


# TFIDF